From: https://github.com/Quorumetrix/Blender_scripts/blob/main/Mesh%20Decimation%20Pipeline.ipynb

In [ ]:
import pandas as pd
import numpy as np
import os
import time
import h5py
import pyvista as pv
from tqdm import tqdm

In [ ]:
pd.set_option('display.max_columns', None)

# this csv file is 268 MB and too large to place on Github
# in order to run this notebook, you must first download the file and place on your local machine in the /data folder
# the csv file is available here: https://zenodo.org/record/5579388/files/211019_mitochondria_info.csv

mito = pd.read_csv('data/211019_mitochondria_info.csv')
# mito

astrocytes = pd.read_csv('data/astrocytes.csv', index_col=[0])
# astrocytes.head()

astrocyte_list = astrocytes.cell_segid.to_list()
len(astrocyte_list)

In [ ]:
mito[mito.cellid == astrocyte_list[0]].mito_id.to_list()

In [ ]:
# astrocyte_list.remove(648518346349527319)

In [ ]:
len(astrocyte_list)

In [ ]:
# import os
# import time
# import h5py
# import numpy as np
# import pyvista as pv
# import pandas as pd
# from tqdm import tqdm

# Set up paths and parameters
obj_path = 'data/meshes/'
dec_prcnt = 95
dec_path = 'data/meshes/dec/'

def read_h5_file(file_path):
    with h5py.File(file_path, 'r') as f:
        vertices = np.array(f['vertices'])
        faces = np.array(f['faces'])
    return vertices, faces

def create_pyvista_mesh(vertices, faces):
    # PyVista expects faces to be in the format (n, v1, v2, v3, ...)
    faces_with_size = np.hstack([np.full((faces.shape[0], 1), faces.shape[1]), faces])
    return pv.PolyData(vertices, faces_with_size)

def process_obj(filename, dec_prcnt, obj_path, dec_path, index, total):
    print(f"Processing mito {index + 1} of {total}")
    
    # Load the file
    vertices, faces = read_h5_file(filename)
    
    # Create PyVista mesh
    mesh = create_pyvista_mesh(vertices, faces)
    
    # Perform decimation
    simplified_mesh = mesh.decimate(target_reduction=dec_prcnt / 100)  # Reduce to x% of the original mesh
    
    # Ensure the output directory exists
    if not os.path.exists(dec_path):
        os.makedirs(dec_path)
    
    # Save the simplified mesh in PLY format
    output_filename = os.path.basename(filename).replace('.h5', '.ply')
    output_path = os.path.join(dec_path, output_filename)
    
    print(f"Saving decimated mesh file for {output_filename}")
    try:
        simplified_mesh.save(output_path)
    except Exception as e:
        print(f"Error saving {output_path}: {e}")

# # Iterate through astrocyte_list
# astrocyte_list = [...]  # Define or load your astrocyte_list here
# mito = pd.DataFrame(...)  # Define or load your mito DataFrame here

for astrocyte_id in astrocyte_list:
    mito_list = mito[mito.cellid == astrocyte_id].mito_id.to_list()
    total_mitos = len(mito_list)
    with tqdm(mito_list, desc=f"Processing Astrocyte {astrocyte_id}") as pbar:
        for index, mitoid in enumerate(pbar):
            filename = os.path.join(obj_path, str(mitoid) + '.h5')
            process_obj(filename, dec_prcnt, obj_path, dec_path, index, total_mitos)
